##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Интегрированные градиенты

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/interpretability/integrated_gradients"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/interpretability/integrated_gradients.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/interpretability/integrated_gradients.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/interpretability/integrated_gradients.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
  <td>
    <a href="https://tfhub.dev/google/imagenet/inception_v1/classification/4"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />Смотрите модели TF Hub</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом руководстве показано, как реализовать **Integrated Gradients(IG)**, метод [Объясняемого искусственного интеллекта](https://en.wikipedia.org/wiki/Explainable_artificial_intelligence), представленный в статье [Axiomatic Attribution for Deep Networks](https://arxiv.org/abs/1703.01365). `IG` стремится объяснить взаимосвязь между предсказаниями модели с точки зрения ее характеристик. Он имеет множество вариантов использования, включая понимание важности функций, определение перекоса данных и отладку эффективности модели.

`IG` стал популярным методом интерпретируемости из-за его широкой применимости к любой дифференцируемой модели (например изображение, текст, структурированные данные), простоты реализации, теоретического обоснования и вычислительной эффективности по сравнению с альтернативными подходами, которые позволяют масштабировать его до крупных сетей и пространств признаков, таких как изображения.

В этом руководстве вы шаг за шагом пройдете через реализацию `IG`, чтобы понять важность признаков пикселей для классификатора изображений. В качестве примера рассмотрим это [изображение](https://commons.wikimedia.org/wiki/File:San_Francisco_fireboat_showing_off.jpg) с пожарным катером, распыляющим струи воды. Вы бы классифицировали это изображение как пожарную лодку и могли бы выделить пиксели, из которых состоит лодка и водометы, как важные для вашего решения. Позже в этом руководстве ваша модель также классифицирует это изображение как пожарную лодку, однако определеяет ли она те же самые пиксели как важные, при объяснении своего решения?

На изображениях ниже под заголовками `IG Attribution Mask` и `Original + IG Mask Overlay` вы можете видеть, что ваша модель в своем решении выделяет(фиолетовым цветом) пиксели, составляющие водометы и струи воды, как более важные, чем сама лодка. Как ваша модель будет обобщать новые пожарные лодки? А как насчет пожарных катеров без водометов? Читайте дальше, чтобы узнать о том, как работает `IG` и как применить `IG` к вашим моделям.

![изображение 1](images/IG_fireboat.png)

## Установка

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

### Загрузка предварительно обученного классификатора изображений с TF-Hub

IG может применяться к любой дифференцируемой модели. Следуя оригинальной статье, вы будете использовать предварительно обученную версию такой же модели, Inception V1, которую вы загрузите с [TensorFlow Hub](https://tfhub.dev/google/imagenet/inception_v1/classification/4).

In [ ]:
model = tf.keras.Sequential([
    hub.KerasLayer(
        name='inception_v1',
        handle='https://tfhub.dev/google/imagenet/inception_v1/classification/4',
        trainable=False),
])
model.build([None, 224, 224, 3])
model.summary()

Работая с этим руководством, вам необходимо помнить следующее о Inception V1:

**Входные данные**: ожидаемая размерность входных данных для модели - `(None, 224, 224, 3)`. Это 4D тензор содержащий данные типа `float32` и имеющий размерность `(batch_size, height, width, RGB channels)`, элементы которого представляют собой значения RGB цветов пикселей, нормализованные в диапазоне [0, 1]. Первый элемент - `None` показывает, что модель может принимать любой целочисленный размер пакета.

**Выходы**: `tf.Tensor` размерности `(batch_size, 1001)`. Каждая строка представляет собой прогнозируемую оценку модели для каждого из 1001 класса из ImageNet. Для получения индекса предсказанного класса, имеющего наибольшее значение вероятности, вы можете использовать `tf.argmax(predictions, axis = -1)`. Кроме того, вы также можете преобразовать полный вывод модели в предсказанные вероятности для всех классов, используя `tf.nn.softmax(predictions, axis = -1)`, например для количественной оценки неопределенности модели, или для исследования предсказанных классов в процессе отладки.

In [ ]:
def load_imagenet_labels(file_path):
  labels_file = tf.keras.utils.get_file('ImageNetLabels.txt', file_path)
  with open(labels_file) as reader:
    f = reader.read()
    labels = f.splitlines()
  return np.array(labels)

In [ ]:
imagenet_labels = load_imagenet_labels('https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')

### Загрузка и предварительная обработка изображений с помощью `tf.image`

Вы протестируете IG, используя два изображения из [Wikimedia Commons](https://commons.wikimedia.org/wiki/Main_Page): 
[Пожарный катер](https://commons.wikimedia.org/wiki/File:San_Francisco_fireboat_showing_off.jpg ) и 
[Гигантская панда](https://commons.wikimedia.org/wiki/File:Giant_Panda_2.JPG).

In [ ]:
def read_image(file_name):
  image = tf.io.read_file(file_name)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize_with_pad(image, target_height=224, target_width=224)
  return image

In [ ]:
img_url = {
    'Fireboat': 'http://storage.googleapis.com/download.tensorflow.org/example_images/San_Francisco_fireboat_showing_off.jpg',
    'Giant Panda': 'http://storage.googleapis.com/download.tensorflow.org/example_images/Giant_Panda_2.jpeg',
}

img_paths = {name: tf.keras.utils.get_file(name, url) for (name, url) in img_url.items()}
img_name_tensors = {name: read_image(img_path) for (name, img_path) in img_paths.items()}

In [ ]:
plt.figure(figsize=(8, 8))
for n, (name, img_tensors) in enumerate(img_name_tensors.items()):
  ax = plt.subplot(1, 2, n+1)
  ax.imshow(img_tensors)
  ax.set_title(name)
  ax.axis('off')
plt.tight_layout()

### Классификация изображений
Начнем с классификации этих изображений и отображения трех самых надежных прогнозов. Ниже приводится вспомогательная функция для получения топ k прогнозируемых меток и вероятностей.

In [ ]:
def top_k_predictions(img, k=3):
  image_batch = tf.expand_dims(img, 0)
  predictions = model(image_batch)
  probs = tf.nn.softmax(predictions, axis=-1)
  top_probs, top_idxs = tf.math.top_k(input=probs, k=k)
  top_labels = imagenet_labels[tuple(top_idxs)]
  return top_labels, top_probs[0]

In [ ]:
for (name, img_tensor) in img_name_tensors.items():
  plt.imshow(img_tensor)
  plt.title(name, fontweight='bold')
  plt.axis('off')
  plt.show()

  pred_label, pred_prob = top_k_predictions(img_tensor)
  for label, prob in zip(pred_label, pred_prob):
    print(f'{label}: {prob:0.1%}')

## Расчет интегрированных градиентов

Ваша модель, Inception V1, представляет собой обученную функцию, которая описывает связи между входными признаками, значениями пикселей изображения и выходными данными, определяемыми величиной вероятности между 0 и 1. Ранние методы интерпретируемости для нейронных сетей определяют важность признаков используя градиенты, которые говорят вам, какие пиксели имеют самую большую величину прогноза вашей модели в данной точке функции прогнозирования. Однако градиенты описывают только *локальные* изменения функции прогнозирования и не описывают полностью всю функцию. По мере того как ваша модель «изучает» взаимосвязи между отдельным пикселем и правильным классом ImageNet, градиент для этого пикселя будет *понижаться*, то есть становиться все более маленьким и даже приближаться к нулю. Рассмотрим простую функцию модели:

In [ ]:
def f(x):
  """Упрощенная функция модели."""
  return tf.where(x < 0.8, x, 0.8)

def interpolated_path(x):
  """Прямой путь."""
  return tf.zeros_like(x)

x = tf.linspace(start=0.0, stop=1.0, num=6)
y = f(x)

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax0 = fig.add_subplot(121)
ax0.plot(x, f(x), marker='o')
ax0.set_title('Gradients saturate over F(x)', fontweight='bold')
ax0.text(0.2, 0.5, 'Gradients > 0 = \n x is important')
ax0.text(0.7, 0.85, 'Gradients = 0 \n x not important')
ax0.set_yticks(tf.range(0, 1.5, 0.5))
ax0.set_xticks(tf.range(0, 1.5, 0.5))
ax0.set_ylabel('F(x) - model true class predicted probability')
ax0.set_xlabel('x - (pixel value)')

ax1 = fig.add_subplot(122)
ax1.plot(x, f(x), marker='o')
ax1.plot(x, interpolated_path(x), marker='>')
ax1.set_title('IG intuition', fontweight='bold')
ax1.text(0.25, 0.1, 'Accumulate gradients along path')
ax1.set_ylabel('F(x) - model true class predicted probability')
ax1.set_xlabel('x - (pixel value)')
ax1.set_yticks(tf.range(0, 1.5, 0.5))
ax1.set_xticks(tf.range(0, 1.5, 0.5))
ax1.annotate('Baseline', xy=(0.0, 0.0), xytext=(0.0, 0.2),
             arrowprops=dict(facecolor='black', shrink=0.1))
ax1.annotate('Input', xy=(1.0, 0.0), xytext=(0.95, 0.2),
             arrowprops=dict(facecolor='black', shrink=0.1))
plt.show();

* **слева**: градиенты вашей модели для пикселя `x` положительные в диапазоне от 0,0 до 0,8, и равные 0,0 в диапазоне от 0,8 до 1,0. Пиксель `x` явно оказывает значительное влияние на приближение вашей модели к 80% точности прогнозирования. *Имеет ли значение, что важность пикселя `x` мала или непостоянна?*

* **справа**: Интуиция, лежащая в основе IG, накапливает локальные градиенты пикселя `x` для оценки его важности, чтобы определить, сколько он добавляет или вычитает из общей вероятности выходного класса вашей модели. Вы можете разбить вычисление IG на 3 части:
  1. интерполировать небольшие шаги по прямой линии в пространстве функций между 0(базовая линия или начальная точка) и 1(значение входного пикселя)
  2. вычислять градиенты на каждом шаге между предсказаниями вашей модели относительно каждого шага
  3. аппроксимировать интеграл между базовой линией и входными данными путем накопления(совокупного среднего) этих локальных градиентов.

Чтобы усилить эту интуицию, вы пройдете через эти 3 части, применив IG к изображению "Пожарный катер".

### Установка базового уровня

Базовая линия - это входное изображение, используемое в качестве отправной точки для расчета важности признаков. Интуитивно вы можете думать о базовой линии как о представлении влияния каждого пикселя на прогноз «пожарной лодки», в случае его отсутствия или присутствия во входном изображении. В результате выбор базовой линии играет центральную роль в интерпретации и визуализации важности признаков пикселя. Дополнительное обсуждение выбора базовой линии см. в разделе «Дальнейшие шаги» в нижней части этого руководства. Здесь вы будете использовать полностью черное изображение, все значения пикселей которого равны нулю.

Другие варианты, с которыми вы можете поэкспериментировать, включают полностью белое изображение или случайное изображение, которое вы можете создать с помощью метода `tf.random.uniform(shape = (224,224,3), minval = 0.0, maxval = 1.0)`.

In [ ]:
baseline = tf.zeros(shape=(224,224,3))

In [ ]:
plt.imshow(baseline)
plt.title("Baseline")
plt.axis('off')
plt.show()

### Перевод формул в код

Формула для интегрированных градиентов выглядит следующим образом:

$IntegratedGradients_{i}(x) ::= (x_{i} - x'_{i})\times\int_{\alpha=0}^1\frac{\partial F(x'+\alpha \times (x - x'))}{\partial x_i}{d\alpha}$

где:

$_{i}$ = признак   
$x$ = вход  
$x'$ = базовая линия   
$\alpha$ = константа для создания шума

На практике вычисление определенного интеграла не всегда возможно численно и может быть затратным в вычислительном отношении, поэтому вы вычисляете следующее численное приближение:

$IntegratedGrads^{approx}_{i}(x)::=(x_{i}-x'_{i})\times\sum_{k=1}^{m}\frac{\partial F(x' + \frac{k}{m}\times(x - x'))}{\partial x_{i}} \times \frac{1}{m}$

где:

$_{i}$ = признак (единичный пиксель)  

$x$ = вход (тензор с изображением)  

$x'$ = baseline (тензор с изображением)  

$k$ = константа для создания шума

$m$ = количество шагов в приближении суммы Римана

$(x_{i}-x'_{i})$ = условие для отличия от базовой линии. Это необходимо для масштабирования интегрированных градиентов и сохранения их в соответствии с исходным изображением. Путь от базового изображения до входа находится в пиксельном пространстве. Поскольку с IG вы интегрируете по прямой(линейное преобразование), это в конечном итоге примерно эквивалентно интегральному члену производной функции интерполированного изображения по отношению к $\alpha$ с необходимым количеством шагов. Интеграл суммирует градиент каждого пикселя, умноженный на изменение пикселя вдоль пути. Проще реализовать эту интеграцию в виде единых шагов от одного изображения к другому, подставив $x: = (x'+ \alpha (x-x'))$. Таким образом, замена переменных дает $dx = (x-x')d\alpha $. Член $(x-x')$ является постоянным и не учитывается в интеграле.

### Интерполяция изображения

$IntegratedGrads^{approx}_{i}(x)::=(x_{i}-x'_{i})\times\sum_{k=1}^{m}\frac{\partial F(\overbrace{x' + \frac{k}{m}\times(x - x')}^\text{interpolate m images at k intervals})}{\partial x_{i}} \times \frac{1}{m}$

Сначала вы создадите [линейную интерполяцию](https://en.wikipedia.org/wiki/Linear_interpolation) между базовой линией и исходным изображением. Вы можете думать об интерполированных изображениях как о небольших шагах в пространстве признаков между базовой линией и вводом, представленными $\alpha$ в исходном уравнении.

In [ ]:
m_steps=50
# Сгенерируем интервалы m_steps для integral_approximation() ниже.
alphas = tf.linspace(start=0.0, stop=1.0, num=m_steps+1) 

In [ ]:
def interpolate_images(baseline,
                       image,
                       alphas):
  alphas_x = alphas[:, tf.newaxis, tf.newaxis, tf.newaxis]
  baseline_x = tf.expand_dims(baseline, axis=0)
  input_x = tf.expand_dims(image, axis=0)
  delta = input_x - baseline_x
  images = baseline_x +  alphas_x * delta
  return images

Давайте используем ф-цию interpolate_images() для генерации интерполированных изображений по линейному пути с альфа-интервалами между черным базовым изображением и примером изображения «Fireboat».

In [ ]:
interpolated_images = interpolate_images(
    baseline=baseline,
    image=img_name_tensors['Fireboat'],
    alphas=alphas)

Визуализируем интерполированные изображения. 

Примечание. Другой способ представить константу $\alpha$ состоит в том, что она последовательно увеличивает интенсивность каждого интерполированного изображения.

In [ ]:
fig = plt.figure(figsize=(20, 20))

i = 0
for alpha, image in zip(alphas[0::10], interpolated_images[0::10]):
  i += 1
  plt.subplot(1, len(alphas[0::10]), i)
  plt.title(f'alpha: {alpha:.1f}')
  plt.imshow(image)
  plt.axis('off')

plt.tight_layout();

### Вычисление градиентов

Теперь давайте посмотрим, как рассчитать градиенты, чтобы измерить взаимосвязь между изменениями в функции и изменениями в прогнозах модели. В случае изображений градиент сообщает нам, какие пиксели имеют наибольшее влияние на вероятности классов, предсказанные моделями.

$IntegratedGrads^{approx}_{i}(x)::=(x_{i}-x'_{i})\times\sum_{k=1}^{m}\frac{\overbrace{\partial F(\text{interpolated images})}^\text{compute gradients}}{\partial x_{i}} \times \frac{1}{m}$

где:  
$F()$ = ф-ция прогнозирования вашей модели  
$\frac{\partial{F}}{\partial{x_i}}$ = градиент(вектор частных производных $\partial$) функции прогнозирования модели F относительно каждого признака $x_i$

TensorFlow упрощает для вас вычисление градиентов с помощью [`tf.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape).

In [ ]:
def compute_gradients(images, target_class_idx):
  with tf.GradientTape() as tape:
    tape.watch(images)
    logits = model(images)
    probs = tf.nn.softmax(logits, axis=-1)[:, target_class_idx]
  return tape.gradient(probs, images)

Давайте вычислим градиенты для каждого изображения вдоль пути интерполяции относительно правильного вывода. Напомним, что ваша модель возвращает тензор размерности `(1, 1001)` с логитами, которые вы конвертируете в предсказанные вероятности для каждого класса. Вам необходимо передать правильный индекс целевого класса ImageNet в функцию `compute_gradients`.

In [ ]:
path_gradients = compute_gradients(
    images=interpolated_images,
    target_class_idx=555)

Обратите внимание на размерность вывода `(n_interpolated_images, img_height, img_width, RGB)`, которая дает нам градиент для каждого пикселя изображения вдоль пути интерполяции. Вы можете представить эти градиенты как величину изменения прогнозов вашей модели для каждого небольшого шага в пространстве функций.

In [ ]:
print(path_gradients.shape)

**Визуализация понижения градиента**

Вспомните, что градиенты, которые вы только что вычислили выше, описывают *локальные* изменения в прогнозируемой вашей моделью вероятности «пожарной лодки» и могут *понижаться*.

Эти концепции визуализируются на двух графиках ниже, с помощиью градиентов, рассчитанных выше .

In [ ]:
pred = model(interpolated_images)
pred_proba = tf.nn.softmax(pred, axis=-1)[:, 555]

plt.figure(figsize=(10, 4))
ax1 = plt.subplot(1, 2, 1)
ax1.plot(alphas, pred_proba)
ax1.set_title('Target class predicted probability over alpha')
ax1.set_ylabel('model p(target class)')
ax1.set_xlabel('alpha')
ax1.set_ylim([0, 1])

ax2 = plt.subplot(1, 2, 2)
# Усредняем по шагам интерполяции
average_grads = tf.reduce_mean(path_gradients, axis=[1, 2, 3])
# Нормализуем градиент до [0, 1]. Например (x - min(x))/(max(x)-min(x))
average_grads_norm = (average_grads-tf.math.reduce_min(average_grads))/(tf.math.reduce_max(average_grads)-tf.reduce_min(average_grads))
ax2.plot(alphas, average_grads_norm)
ax2.set_title('Average pixel gradients (normalized) over alpha')
ax2.set_ylabel('Average pixel gradients')
ax2.set_xlabel('alpha')
ax2.set_ylim([0, 1]);

* **слева**: этот график показывает, как степень уверенности вашей модели в классе «Fireboat» варьируется в зависимости от величины альфа. Обратите внимание на то, как градиенты уменьшаются и наклон линии сглаживается между 0,6 и 1,0, прежде чем значения установятся на вероятности «Fireboat» около 40%.

* **справа**: правый график более точно показывает среднюю величину градиента. Обратите внимание, как значения резко приближаются к нулю и даже ненадолго опускаются ниже нуля. Фактически, ваша модель больше всего "учится" на градиентах при более низких значениях альфа. Это можно представить как ситуацию, когда ваша модель выучила пиксели, например водяных пушек, чтобы сделать правильный прогноз, подавив градиенты этих пикселей до нуля, но все еще довольно неопределенна и сфокусирована на ложных пикселях моста или водяной струи, поскольку значения альфа приближаются к исходному входному изображению.

Чтобы убедиться, что пиксели водяной пушки определены как **важные** для прогноза «Fireboat», ниже вы узнаете как накапливать эти градиенты, чтобы оценить, как каждый пиксель влияет на тчоность узнавания «Fireboat».

### Накопление градиентов (интегральное приближение)

Есть много способов вычисления численного приближения интеграла для IG с различными компромиссами в точности и сходимости для различных функций. Популярный класс методов называется [суммы Римана](https://en.wikipedia.org/wiki/Riemann_sum). Здесь вы будете использовать правило трапеции(вы можете найти дополнительный код для изучения различных методов приближения в конце этого руководства).

$IntegratedGrads^{approx}_{i}(x)::=(x_{i}-x'_{i})\times \overbrace{\sum_{k=1}^{m}}^\text{Sum m local gradients}
\text{gradients(interpolated images)} \times \overbrace{\frac{1}{m}}^\text{Divide by m steps}$

Из уравнения вы можете видеть, что суммируете по `m` градиентов и делите на `m` шагов. Вы можете реализовать две операции 3-й части как *среднее значение локальных градиентов интерполированных предсказаний `m` и входных изображений*.

In [ ]:
def integral_approximation(gradients):
  # riemann_trapezoidal
  grads = (gradients[:-1] + gradients[1:]) / tf.constant(2.0)
  integrated_gradients = tf.math.reduce_mean(grads, axis=0)
  return integrated_gradients

Функция `integral_approximation` берет градиенты предсказанной вероятности целевого класса по отношению к интерполированным изображениям между базовой линией и исходным изображением.

In [ ]:
ig = integral_approximation(
    gradients=path_gradients)

Вы можете убедиться, что усреднение по градиентам интерполированных изображений `m` возвращает интегрированный тензор градиентов той же размерности, что и исходное изображение.

In [ ]:
print(ig.shape)

### Собираем все вместе

Теперь вы объедините три предыдущие части в функцию `IntegratedGradients` и воспользуетесь декоратором [@ tf.function](https://www.tensorflow.org/guide/function), чтобы скомпилировать его в высокопроизводительный граф TensorFlow. Это реализовано в виде 5 небольших шагов ниже:

$IntegratedGrads^{approx}_{i}(x)::=\overbrace{(x_{i}-x'_{i})}^\text{5.}\times \overbrace{\sum_{k=1}^{m}}^\text{4.} \frac{\partial \overbrace{F(\overbrace{x' + \overbrace{\frac{k}{m}}^\text{1.}\times(x - x'))}^\text{2.}}^\text{3.}}{\partial x_{i}} \times \overbrace{\frac{1}{m}}^\text{4.}$

1. Создаем альфа $\alpha$

2. Генерируем интерполированные изображения = $(x '+ \frac{k}{m}\times(x - x'))$

3. Вычисляем градиенты между предсказаниями выходных данных модели $F$ относительно входных признаков = $\frac{\partial F(\text{интерполированные входные пути})}{\partial x_{i}}$

4. Делаем интегральное приближение через усреднение градиентов = $\sum_{k=1}^m \text{градиент} \times \frac{1}{m}$

5. Масштабируем интегрированные градиенты относительно исходного изображения = $(x_{i}-x'_ {i}) \times \text {интегрированные градиенты}$. Причина, по которой этот шаг необходим, состоит в том, что нужно убедиться, что значения атрибуций, накопленные для нескольких интерполированных изображений, находятся в одной и той же размерности и точно отражают важность пикселей на исходном изображении.

In [ ]:
@tf.function
def integrated_gradients(baseline,
                         image,
                         target_class_idx,
                         m_steps=50,
                         batch_size=32):
  # 1. Создаем альфа
  alphas = tf.linspace(start=0.0, stop=1.0, num=m_steps+1)

  # Инициализируем TensorArray вне цикла для сбора градиентов.    
  gradient_batches = tf.TensorArray(tf.float32, size=m_steps+1)
    
  # Итерируемся сразу по альфа и пакетам для улучшения производительности при больших m_steps.
  for alpha in tf.range(0, len(alphas), batch_size):
    from_ = alpha
    to = tf.minimum(from_ + batch_size, len(alphas))
    alpha_batch = alphas[from_:to]

    # 2. Генерация интерполированных входных данных между базовой линией и входом.
    interpolated_path_input_batch = interpolate_images(baseline=baseline,
                                                       image=image,
                                                       alphas=alpha_batch)

    # 3. Вычисление градиентов между выходными данными модели и интерполированными входными данными.
    gradient_batch = compute_gradients(images=interpolated_path_input_batch,
                                       target_class_idx=target_class_idx)
    
    # Записываем пакеты индексов и градиентов во внешний TensorArray.
    gradient_batches = gradient_batches.scatter(tf.range(from_, to), gradient_batch)    
  
  # Складываем градиенты пути построчно в один тензор.
  total_gradients = gradient_batches.stack()

  # 4. Интегральное приближение через усредненные градиенты.
  avg_gradients = integral_approximation(gradients=total_gradients)

  # 5. Масштабирование интегрированных градиентов относительно ввода.
  integrated_gradients = (image - baseline) * avg_gradients

  return integrated_gradients

In [ ]:
ig_attributions = integrated_gradients(baseline=baseline,
                                       image=img_name_tensors['Fireboat'],
                                       target_class_idx=555,
                                       m_steps=240)

И снова, вы можете проверить, что атрибуты объекта IG имеют ту же размерность, что и входное изображение «Fireboat».

In [ ]:
print(ig_attributions.shape)

В документе предлагается количество шагов в диапазоне от 20 до 300 в зависимости от примера(хотя на практике может потребоваться более 1000 шагов, чтобы точно аппроксимировать интеграл). Вы можете найти дополнительный код для проверки необходимого количества шагов в разделе «Следующие шаги» в конце этого руководства.

### Визуализация "мышления" модели

Теперь вы готовы визуализировать процесс "распознования признаков" и наложить его на исходное изображение. В приведенном ниже коде суммируются абсолютные значения интегрированных градиентов по цветовым каналам для создания маски процесса распознования. Этот метод построения графика отражает относительное влияние пикселей на прогнозы модели.

In [ ]:
def plot_img_attributions(baseline,
                          image,
                          target_class_idx,
                          m_steps=50,
                          cmap=None,
                          overlay_alpha=0.4):

  attributions = integrated_gradients(baseline=baseline,
                                      image=image,
                                      target_class_idx=target_class_idx,
                                      m_steps=m_steps)

  # Сумма атрибутов по цветовым каналам для визуализации.
  # Форма маски - это изображение в оттенках серого, 
  # высота и ширина которого равны исходному изображению.
  attribution_mask = tf.reduce_sum(tf.math.abs(attributions), axis=-1)

  fig, axs = plt.subplots(nrows=2, ncols=2, squeeze=False, figsize=(8, 8))

  axs[0, 0].set_title('Baseline image')
  axs[0, 0].imshow(baseline)
  axs[0, 0].axis('off')

  axs[0, 1].set_title('Original image')
  axs[0, 1].imshow(image)
  axs[0, 1].axis('off')

  axs[1, 0].set_title('Attribution mask')
  axs[1, 0].imshow(attribution_mask, cmap=cmap)
  axs[1, 0].axis('off')

  axs[1, 1].set_title('Overlay')
  axs[1, 1].imshow(attribution_mask, cmap=cmap)
  axs[1, 1].imshow(image, alpha=overlay_alpha)
  axs[1, 1].axis('off')

  plt.tight_layout()
  return fig

Глядя на объекты характеристик модели на изображении «Пожарная лодка», вы можете увидеть, что модель определяет водометы и струи как способствующие ее правильному прогнозу.

In [ ]:
_ = plot_img_attributions(image=img_name_tensors['Fireboat'],
                          baseline=baseline,
                          target_class_idx=555,
                          m_steps=240,
                          cmap=plt.cm.inferno,
                          overlay_alpha=0.4)

На изображении «Гигантская панда» объектами характеристик являются текстуры, нос и мех лица панды.


In [ ]:
_ = plot_img_attributions(image=img_name_tensors['Giant Panda'],
                          baseline=baseline,
                          target_class_idx=389,
                          m_steps=55,
                          cmap=plt.cm.viridis,
                          overlay_alpha=0.5)

## Использование и ограничения

Сценарии использования
* Использование таких методов, как интегрированные градиенты, перед развертыванием модели может дать вам понимание того, как и почему работает модель. Соответствуют ли признаки, выделенные с помощью этой техники, вашему пониманию? В противном случае это может указывать на ошибку в вашей модели или наборе данных, или на переобучение.

Ограничения
* Интегрированные градиенты обеспечивают важность признаков на отдельных примерах, однако не предоставляют глобальную важность признаков для всего набора данных.

* Интегрированные градиенты предоставляют индивидуальные значения важности признаков, но не объясняют их взаимодействие и комбинации.

## Следующие шаги

В этом руководстве представлена базовая реализация интегрированных градиентов. В качестве следующего шага вы можете использовать этот ноутбук, чтобы самостоятельно опробовать эту технику с разными моделями и изображениями.

Для заинтересованных читателей есть расширенная версия этого руководства (которая включает код для различных базовых показателей, для вычисления интегральных приближений и определения достаточного количества шагов), которую вы можете найти [здесь](https://github.com/GoogleCloudPlatform/training-data-analyst/tree/master/blogs/Integrated_gradients).

Чтобы углубить свое понимание интегрированных градиентов, ознакомьтесь со статьей [Axiomatic Attribution for Deep Networks](https://arxiv.org/abs/1703.01365) и [Github репозиторием](https://github.com/ankurtaly/Integrated-Gradients), который содержит реализацию в предыдущей версии TensorFlow. Вы также можете изучить атрибуцию признаков и влияние различных базовых показателей на [distill.pub](https://distill.pub/2020/attribution-baselines/).

Заинтересованы во внедрении IG в производственные рабочие процессы машинного обучения для определения важности функций, анализа ошибок модели и мониторинга перекоса данных? Ознакомьтесь с продуктом Google Cloud [Explainable AI](https://cloud.google.com/explainable-ai), который поддерживает атрибуцию IG. Исследовательская группа Google AI PAIR также открыла исходный код [What-if tool](https://pair-code.github.io/what-if-tool/index.html#about), который можно использовать для отладки модели, включая визуализацию "образа мышления" модели IG.